In [ ]:
# Due to the file system setting of the Jupyter Notebook, navigate to the package to use infra
%cd /home/jupyter/quant

## Batch ETL Historical Daily & Hourly Stock Price Data

In [ ]:
from datetime import datetime
import pandas as pd
import pymysql
from yitian.datasource import *

# Set up cloud sql connections
password = ''
connection = pymysql.connect(host=PRIVATE_HOST,
                             user=USER,
                             password=password,
                             db=DATABASE)

In [ ]:
# Grab tickers from dim tables
nasdaq_pd = pd.read_sql("SELECT * FROM quant.nasdaq", connection)
nasdaq_tickers = nasdaq_pd.symbol

nyse_pd = pd.read_sql("SELECT * FROM quant.nyse", connection)
nyse_tickers = nyse_pd.symbol


#### Batch Update Historical Daily & Hourly Stock Price Data into GCP Cloud Storage and mySQL

In [ ]:
# Extract data to GCP cloud storage and load to mySQL

stop_year = 2020
mode = 'update'

for t in nasdaq_tickers:
    ticker = t
    
    with connection.cursor() as cursor:
        cursor.execute(f" DELETE FROM stock_daily WHERE ticker='{ticker}' AND year={str(stop_year)};")
        cursor.execute(f" DELETE FROM stock_hourly WHERE ticker='{ticker}' AND year={str(stop_year)};")
        
        connection.commit()
 
    ticker = t
    
    %run -i 'notebooks/etl/yfinance/batch_etl.py'

In [ ]:
# Close connection
connection.close()

#### Batch Extract, Transform & Load Historical Daily & Hourly Stock Price Data into GCP Cloud Storage and mySQL

In [ ]:
# ETL historical daily * hourly stock data before 2020

stop_year = 2020
mode = 'history'

for t in nasdaq_tickers:
    
    ticker = t
    
    %run -i 'notebooks/etl/yfinance/batch_etl.py'